# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import logging

import core.config as cconfig
import core.dataflow_model.incremental_single_name_model_evaluator as ime
import core.dataflow_model.model_evaluator as modeval
import core.dataflow_model.model_plotter as modplot
import core.dataflow_model.stats_computer as csc
import core.dataflow_model.utils as cdmu
import core.plotting as cplot
import core.statistics as cstati
import helpers.dbg as dbg
import helpers.printing as hprint

In [ ]:
dbg.init_logger(verbosity=logging.INFO)
# dbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

# _LOG.info("%s", env.get_system_signature()[0])

hprint.config_notebook()

# Load features

In [ ]:
feat_iter = cdmu.yield_experiment_artifacts(
    src_dir="",
    file_name="result_bundle.v2_0.pkl",
    load_rb_kwargs={},
)

In [ ]:
key, artifact = next(feat_iter)
display("key=%s", key)
features = artifact.result_df

In [ ]:
features.head()

# Cross-sectional feature analysis

In [ ]:
cplot.plot_heatmap(
    features.corr(),
    mode="clustermap",
    figsize=(20, 20)
)

In [ ]:
cplot.plot_effective_correlation_rank(features)

In [ ]:
cplot.plot_projection(features.resample("B").sum(min_count=1))

In [ ]:
sc = csc.StatsComputer()

In [ ]:
features.apply(sc.compute_summary_stats).round(3)

# Single feature analysis

In [ ]:
feature = ""

In [ ]:
cplot.plot_qq(features[feature])

In [ ]:
cplot.plot_histograms_and_lagged_scatterplot(
    features[feature],
    lag=1,
    figsize=(20, 20)
)

In [ ]:
cplot.plot_time_series_by_period(
    features[feature],
    "hour",
)